## テレコムサンプルデータジェネレーター
Databricks Labs Data Generatorを使用してテレコム関連のデータを生成

### パラメータ設定

In [0]:
%run ./00_環境設定

### ライブラリインポート

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string

### 指名や地域等の値のパターンを定義

In [0]:
# ランダムデータ生成用のマスターリスト
LAST_NAMES = ['田中', '鈴木', '佐藤', '高橋', '渡辺', '伊藤', '山本', '中村', '小林', '加藤',
              '吉田', '山田', '佐々木', '山口', '松本', '井上', '木村', '林', '斎藤', '清水']
FIRST_NAMES = ['太郎', '花子', '一郎', '美咲', '健太', '由美', '直樹', '愛子', '大輔', '美穂',
               '翔太', '結衣', '和也', 'さくら', '健一', '真由美', '隆', '優子', '修', '恵子']
PREFECTURES = ['東京都', '大阪府', '神奈川県', '愛知県', '埼玉県', '千葉県', '兵庫県', '北海道',
               '福岡県', '静岡県', '茨城県', '広島県', '京都府', '宮城県', '新潟県']
CITIES = ['渋谷区', '新宿区', '港区', '中央区', '品川区', '北区', '中区', '西区', '南区', '東区',
          '札幌市', '横浜市', '名古屋市', '福岡市', '仙台市', '広島市', '京都市']
RADIO_TYPES = ['LTE', '5G', '5G']  # 5Gの比率を高める
BANDS = ['800MHz', '1.7GHz', '2.1GHz', '3.5GHz', '3.7GHz', '4.5GHz']
DST_IPS = ['172.217.26.238', '210.130.1.42', '13.114.40.30', '52.192.72.89',
           '157.240.1.35', '104.244.42.129', '8.8.8.8', '1.1.1.1']
PROTOCOLS = ['TCP', 'UDP']
DST_PORTS = [80, 443, 53, 8080, 22, 21, 25, 110]

# 都道府県の東西日本区分
EAST_JAPAN_PREFECTURES = [
    '北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県',
    '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県',
    '新潟県', '山梨県', '長野県', '静岡県'
]
WEST_JAPAN_PREFECTURES = [
    '富山県', '石川県', '福井県', '岐阜県', '愛知県', '三重県',
    '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県',
    '鳥取県', '島根県', '岡山県', '広島県', '山口県',
    '徳島県', '香川県', '愛媛県', '高知県',
    '福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県'
]

### ヘルパー関数

In [0]:
def generate_imsi():
    """ランダムなIMSIを生成（440 + 2桁 + 9桁の数字）"""
    return f"4402{random.randint(0, 99):02d}{random.randint(100000000, 999999999)}"

def generate_msisdn():
    """ランダムな携帯電話番号を生成"""
    prefix = random.choice(['090', '080', '070'])
    return f"{prefix}{random.randint(10000000, 99999999)}"

def generate_cell_id():
    """ランダムな基地局IDを生成"""
    numbers = ''.join([str(random.randint(0, 9)) for _ in range(5)])
    letters = ''.join([random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ') for _ in range(2)])
    return numbers + letters

def generate_area_code():
    """ランダムなエリアコードを生成"""
    return f"{random.randint(10000, 99999)}"
  
def get_area_region(location):
    """locationから都道府県を抽出し、東日本/西日本を判定"""
    for pref in EAST_JAPAN_PREFECTURES:
        if location.startswith(pref):
            return 'east'
    for pref in WEST_JAPAN_PREFECTURES:
        if location.startswith(pref):
            return 'west'
    return 'unknown'

### 加入者マスタ (subscriber_master) の作成

In [0]:
print(f"加入者マスターを {NUM_SUBSCRIBERS} 件生成中...")

# activation_dateを先に生成
activation_dates = [datetime(2023, 1, 1) + timedelta(days=random.randint(0, 730)) 
                    for _ in range(NUM_SUBSCRIBERS)]

subscriber_data = {
    'imsi': [generate_imsi() for _ in range(NUM_SUBSCRIBERS)],
    'msisdn': [generate_msisdn() for _ in range(NUM_SUBSCRIBERS)],
    'subscriber_name': [f"{random.choice(LAST_NAMES)} {random.choice(FIRST_NAMES)}" 
                       for _ in range(NUM_SUBSCRIBERS)],
    'contract_type': [random.choice(['個人', '法人']) for _ in range(NUM_SUBSCRIBERS)],
    'subscriber_status': [random.choice(['有効', '有効', '有効', '有効', '無効']) for _ in range(NUM_SUBSCRIBERS)],
    'activation_date': activation_dates,
    'expiry_date': [datetime(2026, 1, 1) + timedelta(days=random.randint(0, 1460)) 
                   for _ in range(NUM_SUBSCRIBERS)],
    'subscriber_last_updated': [act_date + timedelta(days=random.randint(1, 365), 
                                          hours=random.randint(0, 23),
                                          minutes=random.randint(0, 59),
                                          seconds=random.randint(0, 59)) 
                     for act_date in activation_dates],
    'operation': ['INSERT'] * NUM_SUBSCRIBERS  # 初回データは全てINSERT
}

subscriber_master = pd.DataFrame(subscriber_data)

### 基地局マスタ (cell_master) の作成

In [0]:
cell_data = {
    'cell_id': [generate_cell_id() for _ in range(NUM_CELLS)],
    'location': [f"{random.choice(PREFECTURES)}{random.choice(CITIES)}" 
                for _ in range(NUM_CELLS)],
    'radio_type': [random.choice(RADIO_TYPES) for _ in range(NUM_CELLS)],
    'band': [random.choice(BANDS) for _ in range(NUM_CELLS)],
    'area_code': [generate_area_code() for _ in range(NUM_CELLS)],
    'cell_status': [random.choice(['running', 'running', 'running', 'maintenance']) 
              for _ in range(NUM_CELLS)],
    'cell_last_updated': [datetime(2024, 1, 1) + timedelta(days=random.randint(0, 680),
                                                       hours=random.randint(0, 23),
                                                       minutes=random.randint(0, 59),
                                                       seconds=random.randint(0, 59)) 
                     for _ in range(NUM_CELLS)]
}

cell_master = pd.DataFrame(cell_data)

# 各cell_idのエリアを判定してマッピングを作成
cell_master['region'] = cell_master['location'].apply(get_area_region)
cell_id_to_region = dict(zip(cell_master['cell_id'], cell_master['region']))

# regionカラムは内部使用のみなので削除
cell_master = cell_master.drop('region', axis=1)

### 通信ログの作成

In [0]:
# 通信ログのレコード数
num_records = NUM_LOGS

# 開始時刻（ランダムな日付を生成）
base_time = datetime(2025, random.randint(1, 12), random.randint(1, 28), 
                     random.randint(0, 23), random.randint(0, 59), random.randint(0, 59))

# ランダムなログを生成
traffic_data = {
    'timestamp': [base_time + timedelta(seconds=random.randint(0, 604800)) for _ in range(num_records)],  # 7日間
    'imsi': [random.choice(subscriber_data['imsi']) for _ in range(num_records)],
    'src_ip': [f"10.{random.randint(10, 250)}.{random.randint(1, 250)}.{random.randint(1, 250)}" 
              for _ in range(num_records)],
    'dst_ip': [random.choice(DST_IPS) for _ in range(num_records)],
    'protocol': [random.choice(PROTOCOLS) for _ in range(num_records)],
    'src_port': [random.randint(30000, 65000) for _ in range(num_records)],
    'dst_port': [random.choice(DST_PORTS) for _ in range(num_records)],
    'cell_id': [random.choice(cell_data['cell_id']) for _ in range(num_records)],
    'session_duration': [random.randint(1, 3600) for _ in range(num_records)],
    'bytes_sent': [random.randint(64, 10000000) for _ in range(num_records)],
    'bytes_received': [random.randint(128, 50000000) for _ in range(num_records)],
    'throughput_mbps': [round(random.uniform(0.5, 200.0), 1) for _ in range(num_records)]
}

network_traffic_log = pd.DataFrame(traffic_data)
network_traffic_log = network_traffic_log.sort_values('timestamp').reset_index(drop=True)

# データ型の確認と設定
network_traffic_log['src_port'] = network_traffic_log['src_port'].astype(int)
network_traffic_log['dst_port'] = network_traffic_log['dst_port'].astype(int)
network_traffic_log['session_duration'] = network_traffic_log['session_duration'].astype(int)
network_traffic_log['bytes_sent'] = network_traffic_log['bytes_sent'].astype(int)
network_traffic_log['bytes_received'] = network_traffic_log['bytes_received'].astype(int)

### 作成したデータの書き出し

In [0]:
import random
import string

def generate_unique_code(length=5):
    chars = string.ascii_lowercase + string.digits
    return ''.join(random.sample(chars, length))

random_suffix = generate_unique_code()

In [0]:
# 加入者マスタ CSVファイル書き出し
subscriber_master.to_csv(f'{subscriber_master_cdf_dir}subscriber_master_{random_suffix}.csv', index=False, encoding='utf-8-sig')

# 通信ログをエリア別 CSVファイル書き出し
network_traffic_log['region'] = network_traffic_log['cell_id'].map(cell_id_to_region)
log_east = network_traffic_log[network_traffic_log['region'] == 'east'].drop('region', axis=1)
log_west = network_traffic_log[network_traffic_log['region'] == 'west'].drop('region', axis=1)

log_east.to_csv(f'{network_traffic_log_east_dir}network_traffic_log_east_{random_suffix}.csv', index=False, encoding='utf-8-sig')
log_west.to_csv(f'{network_traffic_log_west_dir}network_traffic_log_west_{random_suffix}.csv', index=False, encoding='utf-8-sig')

# 基地局マスタ テーブル書き出し
cell_master_sdf = spark.createDataFrame(cell_master)
cell_master_sdf.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(cell_master_source_table)

### データの検証

In [0]:
print("=" * 60)
print("【データ検証】")
print("=" * 60)

# 通信ログのIMSIが全て加入者マスターに存在するか確認
unique_imsi_in_log = set(network_traffic_log['imsi'].unique())
unique_imsi_in_master = set(subscriber_master['imsi'].unique())
imsi_check = unique_imsi_in_log.issubset(unique_imsi_in_master)
print(f"✓ 通信ログの全IMSIが加入者マスターに存在: {imsi_check}")
print(f"  （通信ログで使用されているIMSI数: {len(unique_imsi_in_log)}）")

# 通信ログのCell_IDが全て基地局マスターに存在するか確認
unique_cellid_in_log = set(network_traffic_log['cell_id'].unique())
unique_cellid_in_master = set(cell_master['cell_id'].unique())
cellid_check = unique_cellid_in_log.issubset(unique_cellid_in_master)
print(f"✓ 通信ログの全Cell_IDが基地局マスターに存在: {cellid_check}")
print(f"  （通信ログで使用されているCell_ID数: {len(unique_cellid_in_log)}）")

In [0]:
display(cell_master)